In [1]:
labels = [
    'F',
    'BR',
    'AU',
    'FI',
    'IR',
    'A',
    'L',
    'LF',
    'MN',
    'O',
    'PE',
    'SC',
    'SE',
    'US',
    'PO'
]

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [3]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("../_5_model_selection/Model/BiGRU-Bert/")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("../_5_model_selection/Model/BiGRU-Bert/")

def classify_review(text):
    encoding = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1

    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

    return(predicted_labels)



In [4]:
classify_review(text = "the otp at login is not generted")

['AU']

In [34]:
from IPython.display import display, HTML

# Custom CSS styles
custom_css = """
<style>
    .jp-Output {
        font-size: 20px;
    }
    
    .widget-label {
        font-size: 20px;
    }
    
    .widget-text {
        font-size: 20px !important;
    }
</style>
"""

# Display custom CSS
display(HTML(custom_css))


In [5]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [39]:
from ipywidgets import interact, widgets, Layout

# Create input widget with custom style
text_input = widgets.Text(value='', description='Review:', layout=Layout(width='500px',height='40px', font_size='50px',padding='10px'))

# Create output widget with custom style
output_label = widgets.Label(value='', layout=Layout( font_size='20px',padding='10px'))

# Define function to update output
def update_output(change):
    result = classify_review(change.new)
    output_label.value = f'class: {result}'

# Link input widget to update function
text_input.observe(update_output, names='value')

# Display widgets
display(text_input)
display(output_label)


Text(value='', description='Review:', layout=Layout(height='40px', padding='10px', width='500px'))

Label(value='', layout=Layout(padding='10px'))